In [ ]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer
import imblearn
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler,ADASYN
from imblearn.under_sampling import RandomUnderSampler,EditedNearestNeighbours
import xgboost
import inspect
from collections import defaultdict
from tabpfn import TabPFNClassifier
from tqdm.notebook import tqdm
import sys
sys.path.append('/kaggle/input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

In [ ]:
first_category = train.EJ.unique()[0]
train.EJ = train.EJ.eq(first_category).astype('int')
test.EJ = test.EJ.eq(first_category).astype('int')

In [ ]:
predictor_columns = [n for n in train.columns if n != 'Class' and n != 'Id']
x= train[predictor_columns]
y = train['Class']

In [ ]:
from sklearn.model_selection import StratifiedKFold as sKF, KFold as KF, GridSearchCV
cv_outer = sKF(n_splits = 10, shuffle=True, random_state=42)
cv_med = sKF(n_splits = 7, shuffle=True, random_state=42)
cv_inner = KF(n_splits = 5, shuffle=True, random_state=42)
cv_msk = MultilabelStratifiedKFold(n_splits = 7, shuffle=True, random_state=42)

In [ ]:
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

In [ ]:
class Ensemble():
    def __init__(self):
        self.pca = PCA(n_components = 54,whiten=True,svd_solver = "full", random_state = 42)
#         self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        self.classifiers =[xgboost.XGBClassifier(),TabPFNClassifier(N_ensemble_configurations=64)]
    
    def fit(self,X,y):
        y = y.values
        unique_classes, y = np.unique(y, return_inverse=True)
        self.classes_ = unique_classes
#         first_category = X.EJ.unique()[0]
#         X.EJ = X.EJ.eq(first_category).astype('int')
#         X = self.imputer.fit_transform(X)
        X = self.pca.fit_transform(X)
        for classifier in self.classifiers:
            if classifier==self.classifiers[1]:
                classifier.fit(X,y,overwrite_warning =True)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
#         x = self.imputer.transform(x)
        x= self.pca.transform(x)
        probabilities = np.stack([classifier.predict_proba(x) for classifier in self.classifiers])
        averaged_probabilities = np.mean(probabilities, axis=0)
        class_0_est_instances = averaged_probabilities[:, 0].sum()
        others_est_instances = averaged_probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        return new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 

In [ ]:
def training(x,y):
    outer_results = list()
    best_loss = np.inf
    fold = 0
    folds = 20
    for train_idx,val_idx in tqdm(cv_outer.split(x,y), total = folds):
        fold+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = Ensemble()
        model.fit(x_train, y_train)
        y_pred = model.predict_proba(x_val)
        probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
        p0 = probabilities[:,:1]
        p0[p0 > 0.86] = 1
        p0[p0 < 0.14] = 0
        y_p = np.empty((y_pred.shape[0],))
        for i in range(y_pred.shape[0]):
            if p0[i]>=0.5:
                y_p[i]= False
            else :
                y_p[i]=True
        y_p = y_p.astype(int)
        loss = balanced_log_loss(y_val,y_p)

        if loss<best_loss:
            best_model = model
            best_loss = loss
            print('best_model_saved')
        outer_results.append(loss)
        print('>val_loss=%.5f, fold = %.1f' % (loss,fold))
    print('LOSS: %.5f' % (np.mean(outer_results)))
    return np.mean(outer_results), best_model
    

In [ ]:
# from datetime import datetime
# times = greeks.Epsilon.copy()
# times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
# times[greeks.Epsilon == 'Unknown'] = np.nan

In [ ]:
# greeks.Alpha = greeks.Alpha.map({'A' :0, 'B':1,'G' :2, 'D' :3})

In [ ]:
# yess = []
# for col in train.drop(['Id','Class'],axis=1).columns:
#     f = train[col].corr(greeks.Alpha)
#     t =  train[col].corr(train.Class)
#     if (f >=0.15 and t>=0.15)  or  (f<=-0.15 and t<=-0.15):
#         yess.append(col)

In [ ]:
# len(yess)

In [ ]:
# train_pred_and_time = pd.concat((train.drop(['Id','Class'],axis=1), times), axis=1)
train_pred_and_time = train[predictor_columns]
train_pred_and_time['Class'] = train.Class
test_predictors = test[predictor_columns]
# first_category = test_predictors.EJ.unique()[0]
# test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
test_pred_and_time = test_predictors
# test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)

In [ ]:
ros = ADASYN(random_state=42)
enn = EditedNearestNeighbours()

si = SimpleImputer(missing_values=np.nan, strategy='median')
train_pred_and_time11 = si.fit_transform(train_pred_and_time)
# train_pred_and_time11 = normalize(train_pred_and_time11, axis=0)
train_ros, y_ros = ros.fit_resample(train_pred_and_time11, greeks.Alpha)
# x_ros, y_ros = ros.fit_resample(df.drop(['Class'],axis=1), df.Class)

print('Original dataset shape')
print( greeks.Alpha.value_counts())
print('Resample dataset shape')
print( y_ros.value_counts())

In [ ]:
df = pd.DataFrame(train_pred_and_time11,columns =train_pred_and_time.columns)

In [ ]:
x_ros = df.drop(['Class'],axis=1)
y_ = df.Class

In [ ]:
yt = Ensemble()

In [ ]:
hist = {}
for col in x_ros:
    print("")
    print(f"{col} started")
    loss , m = training(x_ros.drop([col],axis=1),y_)
    hist[col] = loss
    print(f"{col} done")
    print("")

In [ ]:
hist

In [ ]:
# m.fit(df.drop(['Class'],axis=1),y_ros)

In [ ]:
# y_ros.value_counts(normalize = True)

In [ ]:
# y_pred = m.predict_proba(test_pred_and_time)
# probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
# p0 = probabilities[:,:1]
# p0[p0 > 0.86] = 1
# p0[p0 < 0.14] = 0

In [ ]:
# submission = pd.DataFrame(test["Id"], columns=["Id"])
# submission["class_0"] = p0
# submission["class_1"] = 1 - p0
# submission.to_csv('submission.csv', index=False)

In [ ]:
# submission_df = pd.read_csv('submission.csv')
# submission_df